# LSTM _Train
코드참고 : https://github.com/KMJJ1/hiphop/blob/master/test_example.ipynb

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import collections
import numpy as np
import gensim
from gensim.models import Phrases
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

class TextLoader:
    def __init__(self, path):
        with open(path, "r", encoding='utf-8') as _file:
            self.text = _file.read().split()
            # self.text = self.text[:10000]

        self.song2vec = Word2Vec.load("all_word2vec_model")
        self.vocab, self.words = self.build_vocab()

        self.X = self.text[:] # 텍스트 파일 전체를 복사 - self.text 와 같은 의미
        self.y = [self.text[0]] + self.text[1:] 

    def build_vocab(self):
        vocab_inv = list(self.song2vec.wv.vocab.keys()) # key 값을 리스트화 / 글자만
        vocab = {x: i for i, x in enumerate(vocab_inv)} # 0,1,2 등 인덱스와 단어를 dict 로 매칭시켜놓음
        return vocab, vocab_inv

    def next_batch(self, batch_size, seq_length):
        start = np.random.randint(0, len(self.X)-batch_size*seq_length) # 랜덤으로 위치를 정함 - 끝의 값을 구하면 안됨 / 시작 위치를 글자를 다 배치사이즈와 시퀀스렝스로 구함 // 마지막까지는 안가겠다는 뜻
        end   = start + batch_size*seq_length # 몇 단어를 가져올지

        X_words = self.X[start:end]# 말그대로 글자
        y_words = self.y[start:end]

        X_idx = np.empty((batch_size, seq_length), dtype=np.int64) # 글자의 인덱스
        y_idx = np.empty((batch_size, seq_length), dtype=np.int64)
        X_wv = np.empty((batch_size, seq_length, 50)) # 50 = word2vec size 
        y_wv = np.empty((batch_size, seq_length, 50))
        # 위에서 만들어준 자리에 따라 (저장공간 설정하는 과정) 아래에서 for 문을 돌며 값을 가져옴 / 그냥 하면 안되는 이유 : append는 느림, numpy의 경우에는 1 2 3 4 붙어있어야 함 / 5를 넣는다 하면 이걸 어딘가 복사해서 5를 붙여야 함.
        for i in range(batch_size):
            for j in range(seq_length):
                X_idx[i, j] = self.vocab[X_words[i*seq_length+j]]
                y_idx[i, j] = self.vocab[y_words[i*seq_length+j]]

                X_wv[i, j] = self.song2vec.wv[X_words[i*seq_length+j]]
                y_wv[i, j] = self.song2vec.wv[y_words[i*seq_length+j]]

        return X_wv, X_idx, y_wv, y_idx

In [3]:
#!pip install tensorflow

In [4]:
num_layers  = 3
hidden_size = 512
batch_size  = 100
max_length  = 1
learning_rate = 0.001

loader = TextLoader("all_sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [5]:
# with tf.device("/gpu:5"):
X = tf.placeholder(tf.float32, [None, max_length, 50])  # 50 = word2vec size ㅠ
y = tf.placeholder(tf.int32, [None, max_length]) # 인풋은 워드투벡 아웃풋은 단어?

y_one_hot = tf.one_hot(y, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, _ = tf.nn.dynamic_rnn(cells, X, initial_state=initial_state, dtype=tf.float32)

outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.linear(outputs, vocab_size)
y_flat = tf.reshape(y_one_hot, [-1, vocab_size])


loss_op = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_flat)
loss_op = tf.reduce_mean(loss_op)

opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
saver = tf.train.Saver()

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))


with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, "./new_test_model") 


    for step in range(0, 5):
        X_wv, X_idx, y_wv, y_idx = loader.next_batch(batch_size, max_length)
        loss, _ = sess.run([loss_op, opt], feed_dict={X: X_wv, y: y_idx})

        if (step+1) % 1 == 0:
            print("{:08d} step, loss:{:.4f}".format(step+1, loss))

            random = np.random.randint(0, batch_size)
            results = sess.run(pred, feed_dict={X: X_wv})
            words = [loader.words[word] for word in results[random]]
            print(" ".join(words))

#         if (step+1) % 5 == 0: 
#             saver.save(sess, "char-rnn_"+str(step+1)) # 원래 checkpoints/

00000001 step, loss:11.8683
br
00000002 step, loss:11.8673
br
00000003 step, loss:11.8666
br
00000004 step, loss:11.8659
br
00000005 step, loss:11.8638
br
